In [29]:
!pwd

/ekaterina/work/src/lca/lca/notebooks


In [30]:
'''
Assign the log file name.
'''

clustering_file = '/ekaterina/work/src/lca/lca/tmp/zebra_drone/ca_filtered_merged_tracking_ids_dji_0143_0144_no_unwanted_viewpoints_pre_miewid_right/clustering.json'
node2uuid_file = '/ekaterina/work/src/lca/lca/tmp/zebra_drone/ca_filtered_merged_tracking_ids_dji_0143_0144_no_unwanted_viewpoints_pre_miewid_right/node2uuid_file.json'
anno_file = '/ekaterina/work/src/lca/lca/data/drone/ca_filtered_merged_tracking_ids_dji_0143_0144_no_unwanted_viewpoints_pre_miewid.csv.json'

In [31]:
import pandas as pd
import json

def load_json(file_path):
    with open(file_path) as f:
        data = json.load(f)
    return data

data = load_json(anno_file)

dfc = pd.DataFrame(data['categories'])
dfa = pd.DataFrame(data['annotations'])
dfi = pd.DataFrame(data['images'])

df = dfa.merge(dfi, left_on='image_uuid', right_on='uuid')
# df = dfa.merge(dfi, left_on='image_uuid', right_on='uuid')
# df = df.merge(dfc, left_on='id', right_on='id')

# df = df[df['annotations_census'] == True]
df = df[df['viewpoint'] == 'right']

counts = df.groupby('viewpoint').size()
print(counts)


viewpoint
right    3140
dtype: int64


In [32]:
clusters = load_json(clustering_file)
node2uuid = load_json(node2uuid_file)


uuid_to_cluster = {}


for cluster_id, nodes in clusters.items():
    for node in nodes:
        uuid = node2uuid.get(str(node))  # Get UUID from node2uuid
        if uuid:
            uuid_to_cluster[uuid] = cluster_id  # Map UUID to cluster ID
        

print(uuid_to_cluster)
# print(node2uuid)

{'61f8f18d-ed6a-43ca-bb5e-4e77cbca1234': 'tc065', 'dd974d59-1926-4e71-9ef3-7fd990a11b5f': 'tc450', '198a3ea8-e405-44dc-a8a8-5cff9f9872fa': 'tc450', '41c68658-c9eb-4d0a-8fe1-33daca2910ac': 'tc450', 'f845d84e-077f-4efd-b6b5-7973e35c923c': 'tc450', '79c3c343-644f-4cfa-9cc8-b6d300242213': 'tc450', '0fb03689-5e8e-4e54-a3d4-7d7e23758568': 'tc450', 'ae9b090c-56a2-42d4-9492-2342d783134c': 'tc450', 'b9693907-953a-4d95-9743-58c5be6ad8c9': 'tc450', 'cb44d6af-16c0-4b50-bd3f-68b3b0655385': 'tc450', '6d74531a-052e-43b3-ae57-e3e46285e7dd': 'tc450', '9b993423-2f68-4c00-9e9f-fd339c610a4f': 'tc450', 'b27d5127-59f5-4003-818f-39612e484a60': 'tc450', '4490dbf0-f66f-4f8e-ac6b-52f8d5cde969': 'tc450', 'cd43bdfb-b83f-4115-b053-c82aec975757': 'tc450', 'd9bf3076-ec1d-4427-8e26-473437101fc7': 'tc450', '434954e8-5a91-468d-9e5c-21d3393741ef': 'tc450', '4eba88f6-4344-4223-9ffd-3ce22862a1e3': 'tc450', '5cd7ea40-5da0-42e3-ac28-9d22220bf4a4': 'tc450', '3f5740dd-fd6d-4909-a2d5-b78959cb50ea': 'tc450', 'e46753ad-2021-470b

In [33]:
import random
cluster_ids = list(clusters.keys())

# Function to get cluster ID or assign a random cluster if missing
def get_cluster_or_random(uuid):
    return uuid_to_cluster.get(uuid, random.choice(cluster_ids))

# Apply the function to map or assign random values for missing mappings
df['uuid'] = df['uuid_x']
# df['LCA_clustering_id'] = df['uuid'].apply(get_cluster_or_random)

# df['LCA_clustering_id'] = df['uuid'].map(uuid_to_cluster)
df['LCA_clustering_id'] = df['uuid'].map(uuid_to_cluster).where(df['uuid'].isin(uuid_to_cluster), None)


In [34]:
import json
output_path = '/ekaterina/work/src/lca/lca/data/drone/ca_filtered_merged_tracking_ids_dji_0143_0144_no_unwanted_viewpoints_pre_miewid_right_LCA.json'

df_annotations_fields = ['uuid', 'image_uuid', 'bbox', 'viewpoint', 'tracking_id', 'individual_id', 'confidence','detection_class','species', 'CA_score', 'category_id', 'LCA_clustering_id']
df_annotations = df[df_annotations_fields]
# df_annotations = df_annotations.rename(columns={'uuid_x': 'uuid'})


df_images_fields = ['image_uuid', 'file_name']
df_images_fields= df.columns.intersection(df_images_fields)
df_images = df[df_images_fields].drop_duplicates(keep='first').reset_index(drop=True)
df_images = df_images.rename(columns={'image_uuid': 'uuid'})


df_categories_fields = ['category_id', 'species']
df_categories = df[df_categories_fields].drop_duplicates(keep='first').reset_index(drop=True)
df_categories = df_categories.rename(columns={'category_id': 'id'})

result_dict = {
                'categories': df_categories.to_dict(orient='records'),
                'images': df_images.to_dict(orient='records'),
                'annotations': df_annotations.to_dict(orient='records')
                }

with open(output_path, 'w') as f:
    json.dump(result_dict, f, indent=4)
    print('Data is saved to:', output_path)


Data is saved to: /ekaterina/work/src/lca/lca/data/drone/ca_filtered_merged_tracking_ids_dji_0143_0144_no_unwanted_viewpoints_pre_miewid_right_LCA.json
